In [ ]:
import pprint

import geoviews as gv
import panel as pn
import param
import pystac
import rioxarray  # noqa: F401
import xarray as xr
from holoviews.plotting import list_cmaps

import utils.xyt

pn.extension()
gv.extension("bokeh")

In [ ]:
CATALOG_URL = "https://s3.waw3-2.cloudferro.com/swift/v1/wpl-stac/stac/catalog.json"
SITE = "degero"
COLLECTION = "albedo"

In [ ]:
root: pystac.Catalog = pystac.read_file(CATALOG_URL)  # type: ignore
catalog: pystac.Catalog = root.get_child(SITE)  # type: ignore
collection: pystac.Collection = catalog.get_child(COLLECTION)  # type: ignore

In [ ]:
extent = utils.xyt.Extent.from_pystac(collection.extent)

In [ ]:
xyt = utils.xyt.XYT(extent=extent)

In [ ]:
# this is a custom field which provides some default visualization parameters for the zarr datacube
wpl_render = collection.extra_fields["wpl:render"]
pprint.pprint(wpl_render)

In [ ]:
class ZarrDataset(param.Parameterized):
    parent: utils.xyt.XYT = param.ClassSelector(
        class_=utils.xyt.XYT, label="Parent control", allow_None=False, constant=True
    )  # type: ignore

    xy_ds: xr.Dataset = param.ClassSelector(
        class_=xr.Dataset, label="Datacube chunked for spatial reads", allow_None=False, constant=True
    )  # type: ignore
    ts_ds: xr.Dataset = param.ClassSelector(
        class_=xr.Dataset, label="Datacube chunked for temporal reads", allow_None=False, constant=True
    )  # type: ignore

    primary_var_name: str = param.Selector(objects=[])  # type: ignore
    uncertainty_var_name: str | None = param.Selector(objects=[])  # type: ignore
    uncertainty_scalar: float | None = param.Number(default=None, allow_None=True)  # type: ignore

    colormap_name: str = param.Selector(objects=list_cmaps(), allow_None=False)  # type: ignore
    colormap_min: float = param.Number(default=None, allow_None=False)  # type: ignore
    colormap_max: float = param.Number(default=None, allow_None=False)  # type: ignore

    def __init__(self, **params):
        super().__init__(**params)

        data_vars = list(self.xy_ds.data_vars)

        # set primary_var_name options from the Dataset
        self.param.primary_var_name.objects = data_vars
        self.param.primary_var_name.allow_None = False

        # set uncertainty_var_name options from the Dataset
        self.param.uncertainty_var_name.objects = data_vars
        self.param.uncertainty_var_name.allow_None = True

        # reassign values to trigger validation
        if "primary_var_name" in params:
            self.primary_var_name = params["primary_var_name"]
        else:
            self.primary_var_name = data_vars[0]

        if "uncertainty_var_name" in params:
            self.uncertainty_var_name = params["uncertainty_var_name"]
        else:
            self.uncertainty_var_name = None

    @staticmethod
    def from_pystac(parent: utils.xyt.XYT, collection: pystac.Collection) -> "ZarrDataset":
        """Create a ZarrDataset from a pystac Collection."""

        # this is a custom field which provides some default visualization parameters for the zarr datacube
        WPL_RENDER_KEY = "wpl:render"

        if WPL_RENDER_KEY not in collection.extra_fields:
            raise ValueError(f"Collection {collection.id} does not have the required field {WPL_RENDER_KEY}")

        wpl_render = collection.extra_fields[WPL_RENDER_KEY]

        xy_asset_key = next(asset for asset in wpl_render["assets"] if asset.endswith(".xy.zarr"))
        ts_asset_key = next(asset for asset in wpl_render["assets"] if asset.endswith(".ts.zarr"))

        xy_asset = collection.assets[xy_asset_key]
        ts_asset = collection.assets[ts_asset_key]

        xy_ds = xr.open_dataset(
            xy_asset.href,
            **xy_asset.ext.xarray.open_kwargs,  # type: ignore
        )
        ts_ds = xr.open_dataset(
            ts_asset.href,
            **ts_asset.ext.xarray.open_kwargs,  # type: ignore
        )

        return ZarrDataset(
            parent=parent,
            xy_ds=xy_ds,
            ts_ds=ts_ds,
            primary_var_name=wpl_render["primary_var_name"],
            uncertainty_var_name=wpl_render["uncertainty_var_name"],
            uncertainty_scalar=wpl_render["uncertainty_scalar"],
            colormap_name=wpl_render["colormap_name"],
            colormap_min=wpl_render["colormap_range"][0],
            colormap_max=wpl_render["colormap_range"][1],
        )


In [ ]:
obj = ZarrDataset.from_pystac(xyt, collection)

In [ ]:
pn.Param(obj.param, hide_constant=True)